In [1]:
import geopandas as gpd

# Carregar o arquivo GeoJSON
gdf = gpd.read_file('SAD69-96_SHP_estacaometro_point.geojson')

# Verificar o CRS
print("CRS do arquivo:", gdf.crs)

# Descrição dos dados (resumo estatístico)
print(gdf.describe())

# Visualizar as primeiras linhas para entender a estrutura
print(gdf.head())


DataSourceError: SAD69-96_SHP_estacaometro_point.geojson: No such file or directory

In [16]:
import os
import geopandas as gpd
from pyproj import Transformer

In [17]:
# Função para processar GeoJSONs e converter coordenadas
def processar_geojsons_pyproj(arquivos_geojson, pasta_saida='dados_processados', crs_origem='EPSG:29183', crs_destino='EPSG:4326'):
    # Verificar se a pasta de saída existe, senão, criar
    if not os.path.exists(pasta_saida):
        os.makedirs(pasta_saida)
    
    # Configurar o transformador para converter de crs_origem para crs_destino
    transformer = Transformer.from_crs(crs_origem, crs_destino, always_xy=True)

    for arquivo in arquivos_geojson:
        # Carregar o GeoJSON
        gdf = gpd.read_file(arquivo)

        # Criar uma lista para armazenar as novas coordenadas transformadas
        novas_features = []

        # Iterar sobre as features do GeoDataFrame
        for feature in gdf.iterfeatures():
            coords = feature['geometry']['coordinates']

            # Transformar as coordenadas de UTM para latitude/longitude usando pyproj
            longitude, latitude = transformer.transform(coords[0], coords[1])

            # Atualizar a geometria com as novas coordenadas
            feature['geometry']['coordinates'] = [longitude, latitude]

            # Adicionar a feature atualizada à lista
            novas_features.append(feature)

        # Atualizar o GeoDataFrame com as novas coordenadas
        gdf.features = novas_features
        
        # Nome do arquivo de saída
        nome_base = os.path.splitext(os.path.basename(arquivo))[0]
        nome_saida = os.path.join(pasta_saida, f'{nome_base}_EPSG4326.geojson')
        
        # Salvar o arquivo no formato GeoJSON com as coordenadas convertidas
        gdf.to_file(nome_saida, driver='GeoJSON')

        print(f"Arquivo {arquivo} processado e salvo como {nome_saida}")

In [18]:
# Exemplo de uso
arquivos_geojson = ['SAD69-96_SHP_estacaometro_point.geojson']  # Lista dos arquivos a serem processados
processar_geojsons(arquivos_geojson)

CRS do arquivo SAD69-96_SHP_estacaometro_point.geojson diferente do esperado. Transformando de EPSG:4326 para EPSG:4326.
Arquivo SAD69-96_SHP_estacaometro_point.geojson processado e salvo como dados_processados\SAD69-96_SHP_estacaometro_point_EPSG4326.geojson


## verificar as características dos arquivos

In [21]:
import geopandas as gpd

def verificar_crs_arquivos(arquivos_geojson):
    for arquivo in arquivos_geojson:
        # Carregar o GeoJSON
        gdf = gpd.read_file(arquivo)
        
        # Verificar e imprimir o CRS do arquivo
        print(f"Arquivo: {arquivo}")
        print(f"CRS: {gdf.crs}")
        print("-" * 40)

# Exemplo de uso com uma lista de arquivos GeoJSON
arquivos_geojson = [
    'SAD69-96_SHP_areainfluenciametro.geojson',
    'SAD69-96_SHP_corredoronibus_line.geojson',
    'SAD69-96_SHP_estacaometro_point.geojson',
    'SAD69-96_SHP_estacaometroprojeto_point.geojson',
    'SAD69-96_SHP_estacaotrem.geojson',
    'SAD69-96_SHP_estacaotremprojeto_point.geojson',
    'SAD69-96_SHP_ferroviamdc.geojson',
    'SAD69-96_SHP_linhametro_line.geojson',
    'SAD69-96_SHP_linhametroprojeto.geojson',
    'SAD69-96_SHP_linhatrem.geojson',
    'SAD69-96_SHP_linhatremprojeto_line.geojson',
    'SAD69-96_SHP_pontoonibus.geojson',
    'sad6996_terminal_onibus.geojson',]
verificar_crs_arquivos(arquivos_geojson)

Arquivo: SAD69-96_SHP_areainfluenciametro.geojson
CRS: EPSG:4326
----------------------------------------
Arquivo: SAD69-96_SHP_corredoronibus_line.geojson
CRS: EPSG:4326
----------------------------------------
Arquivo: SAD69-96_SHP_estacaometro_point.geojson
CRS: EPSG:4326
----------------------------------------
Arquivo: SAD69-96_SHP_estacaometroprojeto_point.geojson
CRS: EPSG:4326
----------------------------------------
Arquivo: SAD69-96_SHP_estacaotrem.geojson
CRS: EPSG:4326
----------------------------------------
Arquivo: SAD69-96_SHP_estacaotremprojeto_point.geojson
CRS: EPSG:4326
----------------------------------------
Arquivo: SAD69-96_SHP_ferroviamdc.geojson
CRS: EPSG:4326
----------------------------------------
Arquivo: SAD69-96_SHP_linhametro_line.geojson
CRS: EPSG:4326
----------------------------------------
Arquivo: SAD69-96_SHP_linhametroprojeto.geojson
CRS: EPSG:4326
----------------------------------------
Arquivo: SAD69-96_SHP_linhatrem.geojson
CRS: EPSG:4326
---

In [24]:
import os
import json
from pyproj import Transformer

# Função para processar GeoJSON e converter coordenadas
def processar_geojson_pyproj(arquivos_geojson, pasta_saida='dados_processados', crs_origem='EPSG:29183', crs_destino='EPSG:4326'):
    # Verificar se a pasta de saída existe, senão, criar
    if not os.path.exists(pasta_saida):
        os.makedirs(pasta_saida)

    # Configurar o transformador para converter de crs_origem para crs_destino
    transformer = Transformer.from_crs(crs_origem, crs_destino, always_xy=True)

    # Processar cada arquivo na lista
    for arquivo_geojson in arquivos_geojson:
        # Ler o arquivo GeoJSON
        with open(arquivo_geojson, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Verificar se o CRS informado no GeoJSON é diferente do esperado
        if 'crs' in data and data['crs']['properties']['name'] != 'urn:ogc:def:crs:EPSG::4326':
            print(f"Convertendo coordenadas de {crs_origem} para {crs_destino} no arquivo {arquivo_geojson}...")

        # Iterar sobre as features do GeoJSON e converter as coordenadas
        for feature in data['features']:
            if feature['geometry']['type'] == 'Point':
                # Coordenadas originais em UTM
                utm_x, utm_y = feature['geometry']['coordinates']

                # Converter as coordenadas de UTM para latitude/longitude
                lon, lat = transformer.transform(utm_x, utm_y)

                # Atualizar as coordenadas no GeoJSON
                feature['geometry']['coordinates'] = [lon, lat]

        # Nome do arquivo de saída
        nome_base = os.path.splitext(os.path.basename(arquivo_geojson))[0]
        nome_saida = os.path.join(pasta_saida, f'{nome_base}_EPSG4326.geojson')

        # Salvar o arquivo GeoJSON com as coordenadas convertidas
        with open(nome_saida, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"Arquivo {arquivo_geojson} processado e salvo como {nome_saida}")

# Lista dos arquivos a serem processados
arquivos_geojson = [
    'SAD69-96_SHP_areainfluenciametro.geojson',
    'SAD69-96_SHP_corredoronibus_line.geojson',
    'SAD69-96_SHP_estacaometro_point.geojson',
    'SAD69-96_SHP_estacaometroprojeto_point.geojson',
    'SAD69-96_SHP_estacaotrem.geojson',
    'SAD69-96_SHP_estacaotremprojeto_point.geojson',
    'SAD69-96_SHP_ferroviamdc.geojson',
    'SAD69-96_SHP_linhametro_line.geojson',
    'SAD69-96_SHP_linhametroprojeto.geojson',
    'SAD69-96_SHP_linhatrem.geojson',
    'SAD69-96_SHP_linhatremprojeto_line.geojson',
    'SAD69-96_SHP_pontoonibus.geojson',
    'sad6996_terminal_onibus.geojson',
]

# Processar todos os arquivos da lista
processar_geojson_pyproj(arquivos_geojson)


Convertendo coordenadas de EPSG:29183 para EPSG:4326 no arquivo SAD69-96_SHP_areainfluenciametro.geojson...
Arquivo SAD69-96_SHP_areainfluenciametro.geojson processado e salvo como dados_processados\SAD69-96_SHP_areainfluenciametro_EPSG4326.geojson
Convertendo coordenadas de EPSG:29183 para EPSG:4326 no arquivo SAD69-96_SHP_corredoronibus_line.geojson...
Arquivo SAD69-96_SHP_corredoronibus_line.geojson processado e salvo como dados_processados\SAD69-96_SHP_corredoronibus_line_EPSG4326.geojson
Convertendo coordenadas de EPSG:29183 para EPSG:4326 no arquivo SAD69-96_SHP_estacaometro_point.geojson...
Arquivo SAD69-96_SHP_estacaometro_point.geojson processado e salvo como dados_processados\SAD69-96_SHP_estacaometro_point_EPSG4326.geojson
Convertendo coordenadas de EPSG:29183 para EPSG:4326 no arquivo SAD69-96_SHP_estacaometroprojeto_point.geojson...
Arquivo SAD69-96_SHP_estacaometroprojeto_point.geojson processado e salvo como dados_processados\SAD69-96_SHP_estacaometroprojeto_point_EPSG43